# Financial News Summarizer + Sentiment Annotator using Ollama 3.2 (RSS + Full Article)

In [1]:
import os
from dotenv import load_dotenv
import feedparser
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from datetime import datetime
import pandas as pd
import time
import openai

In [2]:
# STEP 1: Set up Ollama client (local model)
ollama_via_openai = openai.OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'
)
MODEL = "llama3.2"

In [3]:
# STEP 2: Parse RSS feed with full article links
RSS_FEED_URL = "https://feeds.finance.yahoo.com/rss/2.0/headline?s=^GSPC&region=US&lang=en-US"

def fetch_rss_entries():
    feed = feedparser.parse(RSS_FEED_URL)
    entries = [(entry.title, entry.link) for entry in feed.entries]
    return entries

In [4]:
# STEP 3: Scrape full article content from URL (basic paragraph grabber)
def fetch_article_content(url):
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with urlopen(req) as response:
            soup = BeautifulSoup(response.read(), 'html.parser')
        paragraphs = soup.find_all('p')
        text = " ".join(p.get_text() for p in paragraphs)
        return text.strip()
    except Exception as e:
        return f"Error fetching article: {e}"

In [11]:
# STEP 4: Use Ollama to summarize + analyze sentiment
def analyze_article_with_ollama(headline, article):
    prompt = (
        f"""
        Headline: {headline}

        Article:
        {article}

        Summarize the article in 3 or 4 lines. 
        The summary must include what the article is about, what its conclusion is and how it arrived at the conclusion.
        Then assess the sentiment as bullish, bearish, or neutral.

        Respond in this exact format:
        Summary: <summary here>
        Sentiment: <bullish/bearish/neutral>
        """
    )
    try:
        response = ollama_via_openai.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

In [12]:
# STEP 5: Parse response
def parse_response(response_text):
    print("--- Raw Ollama Response ---")
    print(response_text)
    print("---------------------------")
    
    summary = ""
    sentiment = ""
    for line in response_text.splitlines():
        if 'summary' in line.lower():
            summary = line.split(":", 1)[-1].strip()
        elif 'sentiment' in line.lower():
            sentiment = line.split(":", 1)[-1].strip().lower()
    return summary, sentiment

In [13]:
# STEP 6: Process full pipeline
def run_pipeline():
    entries = fetch_rss_entries()
    results = []

    for headline, link in entries:
        article = fetch_article_content(link)
        gpt_response = analyze_article_with_ollama(headline, article)
        summary, sentiment = parse_response(gpt_response)
        results.append({
            "date": datetime.now().strftime('%Y-%m-%d'),
            "headline": headline,
            "summary": summary,
            "sentiment": sentiment,
            "url": link
        })
        time.sleep(1.5)

    df = pd.DataFrame(results)
    df.to_csv("financial_news_summary_ollama.csv", index=False)
    print(df.head())

In [14]:
if __name__ == "__main__":
    run_pipeline()

--- Raw Ollama Response ---
Summary: The article discusses the price-to-earnings (P/E) ratio and how investors can use it to assess value in the tech sector, particularly after recent sell-offs which may have eased valuation concerns. The conclusion is that investors can use the P/E ratio as a tool to evaluate investments in the tech sector. This is achieved by analyzing the historical data of companies and their earnings relative to stock prices. The article's expert provides guidance on how to apply this technical analysis.
 
 Sentiment: Bullish
---------------------------
--- Raw Ollama Response ---
Summary: In an interview with NBC's "Meet the Press" with Kristen Welker, President Donald Trump attributed economic success to his policies and blamed the Biden administration for shortcomings. He downplayed concerns about a recession, but acknowledged that some tariffs could have negative effects on consumers.
Sentiment: Bearish
---------------------------
--- Raw Ollama Response ---
S